<a href="https://colab.research.google.com/github/araujofc/PosIFES/blob/main/FabricioCAraujo_aula4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip titanic.zip

Archive:  titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
data = pd.read_csv("train.csv")
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
y =  data['Survived']
X = data.drop('Survived', axis=1)
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
for column in X.columns:
    print(f'{column:>12} {len(set(X[column])):4} {X[column].dtypes}')

 PassengerId  891 int64
      Pclass    3 int64
        Name  891 object
         Sex    2 object
         Age  265 float64
       SibSp    7 int64
       Parch    7 int64
      Ticket  681 object
        Fare  248 float64
       Cabin  148 object
    Embarked    4 object


In [ ]:
indesejadas = ['PassengerId', 'Name','Ticket','Cabin']
Xdrop = X.drop(indesejadas, axis=1)
Xdrop.columns
# criando uma nova variável chamada Xdrop, que contém o DataFrame X excluindo as
# colunas indesejadas: 'PassengerId', 'Name', 'Ticket' e 'Cabin'.
# A função drop é utilizada para remover colunas (ou linhas) do DataFrame.

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [ ]:
Xnum = Xdrop.select_dtypes('number')
Xnum.columns
for column in Xnum.columns:
    print(f'{column:>12} {sum(Xnum[column].isnull())}')

    #  selecionando colunas numéricas do DataFrame Xdrop e, em seguida,
    #   imprimindo o nome da coluna e a contagem de valores nulos (NaN) para cada coluna numérica.
    # Então, para cada coluna numérica em Xnum, o código imprimirá o nome da
    # coluna e a contagem de valores nulos nessa coluna.

      Pclass 0
         Age 177
       SibSp 0
       Parch 0
        Fare 0


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
XnumLimpo = imputer.fit_transform(Xnum)
XnumLimpo

# a classe SimpleImputer do scikit-learn para preencher os valores nulos (NaN)
# nas colunas numéricas do DataFrame Xnum usando a estratégia da mediana
# A variável resultante XnumLimpo é um novo conjunto de dados numéricos com os valores nulos preenchidos pela mediana.

array([[ 3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    , 28.    ,  1.    ,  2.    , 23.45  ],
       [ 1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 3.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [ ]:
Xcat = Xdrop.select_dtypes('object')
Xcat.columns
for column in Xcat.columns:
    print(f'{column:>12} {sum(Xcat[column].isnull())}')

    #  selecionando as colunas do tipo 'object' (geralmente colunas categóricas) do DataFrame Xdrop e,
    #  em seguida, imprimindo o nome da coluna e a contagem de valores nulos (NaN) para cada coluna categórica.

         Sex 0
    Embarked 2


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
XcatLimpo = imputer.fit_transform(Xcat)
XcatLimpo

#  a classe SimpleImputer do scikit-learn para preencher os valores nulos (NaN) nas colunas categóricas
#   do DataFrame Xcat usando a estratégia do valor mais frequente (most frequent).

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
XcatHot = encoder.fit_transform(XcatLimpo)
XcatHot
import numpy as np
Xtratado = np.c_[XnumLimpo, XcatHot.toarray()]
Xtratado.shape

(891, 10)

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.columns.isin(test.columns)

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
train.columns[~train.columns.isin(test.columns)]

Index(['Survived'], dtype='object')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosDesejados (BaseEstimator, TransformerMixin):
    def fit (self, X, y=None):
        self.colunasIndesejadas_ = ['PassengerId', 'Name','Ticket','Cabin']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas_, axis=1)


atributosDesejados = AtributosDesejados()
Xdrop =  atributosDesejados.fit_transform(X)
Xdrop.columns

#  classe chamada AtributosDesejados, que herda das classes BaseEstimator e TransformerMixin do scikit-learn.
#   Essa classe é projetada para realizar a transformação desejada nos atributos de um conjunto de dados

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [ ]:
from sklearn.base  import  BaseEstimator, TransformerMixin

class AtributosNumericos (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self

    def transform(self, X, y=None):
        return X[self.colunasNumericas]

atributosNumericos = AtributosNumericos()
Xnum = atributosNumericos.fit_transform(Xdrop)
Xnum.columns

#  classe chamada AtributosNumericos, que herda das classes BaseEstimator e TransformerMixin do scikit-learn.
#  Esta classe é projetada para selecionar apenas os atributos numéricos de um conjunto de dados.

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
# cria um pipeline usando a classe Pipeline do scikit-learn.
# Esse pipeline realiza uma sequência de transformações nos atributos numéricos do conjunto de dados.


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('atributos_numericos', AtributosNumericos()),
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler())
])

XnumLimpo = pipeline.fit_transform(Xdrop)
XnumLimpo

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

In [ ]:
from sklearn.base  import BaseEstimator,TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self

    def transform(self, X, y=None):
        return X[self.colunasCategoricas]

atributosCategoricos = AtributosCategoricos()
Xcat= atributosCategoricos.fit_transform(Xdrop)
Xcat.columns


# O código que você forneceu define uma classe chamada AtributosCategoricos,
#  que herda das classes BaseEstimator e TransformerMixin do scikit-learn.
#  Essa classe é projetada para selecionar apenas os atributos categóricos de um conjunto de dados.

Index(['Sex', 'Embarked'], dtype='object')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('atributos_categoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

XcatLimpo = pipecat.fit_transform(Xdrop)
XcatLimpo.toarray()


# O código que você forneceu cria um pipeline para lidar com atributos
#  categóricos usando a classe Pipeline do scikit-learn. Este pipeline realiza
#   uma sequência de transformações nos atributos categóricos do conjunto de dados.

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [ ]:
from sklearn.pipeline import FeatureUnion

unecaracteristicas = FeatureUnion([
    ('pipeline', pipeline),
    ('pipecat', pipecat)
])

Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado.toarray()

# a classe FeatureUnion para combinar características provenientes de dois pipelines
#  diferentes (pipeline e pipecat). A explicação é essencialmente a mesma.
#  A classe FeatureUnion é usada para unir características provenientes de diferentes
#   transformações de maneira paralela.Aqui estamos aplicando a união de características aos dados Xdrop,
#   que passa por dois pipelines distintos (pipeline para características numéricas e pipecat
#   para características categóricas).
#   O resultado é armazenado em Xtratado, e converte a representação
#   esparsa resultante para uma matriz densa usando toarray().

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [ ]:
from sklearn.pipeline import Pipeline

preproc = Pipeline([
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(X)
Xtratado.toarray()

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

pipetotal = Pipeline([
    ('preproc', preproc),
    ('arvore', DecisionTreeClassifier())
])

pipetotal.fit(X, y)
ypred = pipetotal.predict(X)
accuracy_score(y, ypred)

0.9797979797979798

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([0.01543736, 0.01063037, 0.01108456, 0.01056623, 0.0110507 ]),
  'score_time': array([0.00453758, 0.00456285, 0.00437331, 0.00472736, 0.00548673]),
  'test_score': array([0.75977654, 0.80337079, 0.80898876, 0.74157303, 0.82022472])},
 0.7867867679367271)

In [ ]:
pipetotal.fit(X, y)
ypred = pipetotal.predict(test)

submission = pd.read_csv('gender_submission.csv')
submission['Survived'] = ypred
submission.to_csv('tree_submission.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'arvore__max_depth': [None, 1, 3, 5, 7, 9],
    'preproc__unecaracteristicas__pipeline__imputer__strategy': ['mean', 'median', 'most_frequent']
}

modelo = GridSearchCV(pipetotal, param_grid=parametros)

scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.57303762, 1.94370461, 1.54772711, 1.45177341, 1.46605945]),
  'score_time': array([0.00575376, 0.00664473, 0.00464201, 0.00432634, 0.00411248]),
  'test_score': array([0.81564246, 0.82022472, 0.81460674, 0.78089888, 0.83707865])},
 0.8136902893729208)

In [ ]:
modelo.fit(X, y)
modelo.best_estimator_

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('atributos_desejados', AtributosDesejados()),
                                 ('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipeline',
                                                                  Pipeline(steps=[('atributos_numericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipeline(steps=[('atributos_categoricos',
                                                                                   AtributosCategoricos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('encoder',
                                                                                   OneHotEncoder())]))]))])),
                ('arvore', DecisionTreeClassifier(max_depth=5))])